<a href="https://colab.research.google.com/github/AdityaArakere/Sentiment-analysis-model/blob/main/TFModelFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import string
import re
from google.colab import files


uploaded = files.upload()


Saving combined_dataset12.csv to combined_dataset12.csv


In [ ]:
import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded['combined_dataset12.csv']))
df = df[['text', 'sentiment']]

# Preprocess the dataset
# Convert sentiment labels to 0, 1, -1
# df['category'] = df['category'].map({-1: [1, 2], 0: [3, 0], 1: [4, 5]})

# Convert text to lowercase
df['text'] = df['text'].str.lower()

# Remove punctuation
translator = str.maketrans('', '', string.punctuation)
df['text'] = df['text'].astype(str).apply(lambda x: x.translate(translator))

# Remove numbers
df['text'] = df['text'].apply(lambda x: re.sub(r'\d+', '', x))

# Remove extra whitespaces
df['text'] = df['text'].apply(lambda x: ' '.join(x.split()))

# Remove stopwords (optional)
# You can use NLTK or other libraries to remove stopwords if desired

# Print the preprocessed dataset
print(df.head())


                                                text  sentiment
0  i dont know how or why this film has a meager ...          1
1  for a long time it seemed like all the good ca...          1
2  terry gilliams and david peoples teamed up to ...          1
3  what is there to say about an antiestablishmen...          1
4  this movie was made only years after the end o...          1


In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
!pip install transformers
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalMaxPool1D
from tensorflow.keras.callbacks import EarlyStopping

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.4 MB/s eta 0:00:00


In [ ]:
# Split the dataset into train and test sets
x = df['text'].values
y = df['sentiment'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=60000, test_size=12000, random_state=42)

# Print the sizes of the train and test sets
print("Train set size:", len(x_train))
print("Test set size:", len(x_test))
print(y_train[0:5])

Train set size: 60000
Test set size: 12000
[2 1 1 2 1]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define a custom dataset class
class RedditDataset(tf.keras.utils.Sequence):
    def __init__(self, texts, labels, tokenizer, batch_size):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.batch_size = batch_size

        self.on_epoch_end()

    def __len__(self):
        return len(self.texts) // self.batch_size

    def __getitem__(self, index):
        batch_texts = self.texts[index*self.batch_size:(index+1)*self.batch_size]
        batch_labels = self.labels[index*self.batch_size:(index+1)*self.batch_size]

        batch_encodings = self.tokenizer.batch_encode_plus(
            batch_texts,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            max_length=128,
            return_tensors='tf'
        )

        input_tensors = {
            'input_ids': batch_encodings['input_ids'],
            'attention_mask': batch_encodings['attention_mask']
        }

        return input_tensors, batch_labels

    def on_epoch_end(self):
        pass

# Set the hyperparameters
num_epochs = 5
batch_size = 16

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load and preprocess the data
train_dataset = RedditDataset(x_train, y_train, tokenizer, batch_size=batch_size)
test_dataset = RedditDataset(x_test, y_test, tokenizer, batch_size=batch_size)

# Load the pre-trained BERT model for sentiment analysis
bertModel = TFBertModel.from_pretrained('bert-base-uncased')

# Get the BERT embeddings
input_ids = Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='attention_mask')
embeddings = bertModel.bert(input_ids, attention_mask)[0]

# Apply global max pooling
out = tf.keras.layers.GlobalMaxPooling1D()(embeddings)

# Apply dense layers
# out = Dense(128, activation='relu')(out)
# out = tf.keras.layers.Dropout(0.5)(out)
out = Dense(64, activation='relu')(out)
out = tf.keras.layers.Dropout(0.3)(out)

# Create the final output layer
y = Dense(3, activation='softmax')(out)

# Create the model with the defined inputs and outputs
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=y)

# Make the desired layer trainable
model.layers[2].trainable = True

# Define the optimizer
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=5e-5, decay=0.01)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

early_stopping = EarlyStopping(monitor='val_loss', patience=2)

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Train the model
model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset, callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 1/5


3750/3750 [==============================] - 1733s 458ms/step - loss: 0.7015 - accuracy: 0.6962 - val_loss: 0.6282 - val_accuracy: 0.7268
Epoch 2/5
3750/3750 [==============================] - 1748s 466ms/step - loss: 0.6025 - accuracy: 0.7456 - val_loss: 0.6125 - val_accuracy: 0.7345
Epoch 3/5
3750/3750 [==============================] - 1717s 458ms/step - loss: 0.5761 - accuracy: 0.7574 - val_loss: 0.6089 - val_accuracy: 0.7368
Epoch 4/5
3750/3750 [==============================] - 1716s 458ms/step - loss: 0.5654 - accuracy: 0.7635 - val_loss: 0.6028 - val_accuracy: 0.7408
Epoch 5/5
750/750 [==============================] - 114s 152ms/step - loss: 0.6006 - accuracy: 0.7409
Test Loss: 0.6006
Test Accuracy: 74.09%


In [ ]:
# Define the paths to save the model and tokenizer
model_path = '/content/drive/MyDrive/Models/tf_fine_tuned_model12'
tokenizer_path = '/content/drive/MyDrive/Models/tokenizer12'

# Save the fine-tuned model
model.save(model_path)

# Save the tokenizer
tokenizer.save_pretrained(tokenizer_path)


('/content/drive/MyDrive/Models/tokenizer12/tokenizer_config.json',
 '/content/drive/MyDrive/Models/tokenizer12/special_tokens_map.json',
 '/content/drive/MyDrive/Models/tokenizer12/vocab.txt',
 '/content/drive/MyDrive/Models/tokenizer12/added_tokens.json')

In [ ]:
loaded_model = tf.keras.models.load_model("/content/drive/MyDrive/Models/tf_fine_tuned_model11.1")

tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/Models/tokenizer11.1')

# Example custom input
custom_input = "I'm really frustrated with your service. I've been experiencing frequent disruptions and slow response times, which are causing major inconveniences for my work. We apologize for the inconvenience you're facing. Our team is committed to resolving any issues promptly. Can you please provide more details about the disruptions and slow response times you're experiencing? We'll do our best to address the problem and ensure a better experience for you. I've already reached out to your customer support multiple times, but I haven't received a satisfactory resolution. It's frustrating when my concerns aren't being properly addressed. We're sorry to hear about your negative experience with our customer support. This is not the level of service we aim to provide. We assure you that we take your concerns seriously, and we'll investigate the matter to ensure better communication and resolution moving forward. We appreciate your feedback, as it helps us identify areas for improvement. I hope you can address these issues soon. I really want to see improvements in your service, as I believe in the potential of your product. It would be great if I could rely on a more reliable and efficient service in the future."

# Tokenize the custom input
custom_input_encodings = tokenizer.encode_plus(
    custom_input,
    add_special_tokens=True,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='tf'
)

# Get the input IDs
input_ids = custom_input_encodings['input_ids']
attention_mask = custom_input_encodings['attention_mask']

# print(input_ids)

# Get the predictions for the custom input
custom_predictions = loaded_model.predict({'input_ids': input_ids, 'attention_mask': attention_mask})

print(custom_predictions)

# Process the predictions
sentiment_label = tf.argmax(custom_predictions, axis=1)[0]
if sentiment_label == 0:
    sentiment = "Negative"
elif sentiment_label == 1:
    sentiment = "positive"
else:
    sentiment = "neutral"

print(f"Input: {custom_input}")
print(f"Sentiment: {sentiment}")


1/1 [==============================] - 4s 4s/step
[[0.82648736 0.00647429 0.16703837]]
Input: I'm really frustrated with your service. I've been experiencing frequent disruptions and slow response times, which are causing major inconveniences for my work. We apologize for the inconvenience you're facing. Our team is committed to resolving any issues promptly. Can you please provide more details about the disruptions and slow response times you're experiencing? We'll do our best to address the problem and ensure a better experience for you. I've already reached out to your customer support multiple times, but I haven't received a satisfactory resolution. It's frustrating when my concerns aren't being properly addressed. We're sorry to hear about your negative experience with our customer support. This is not the level of service we aim to provide. We assure you that we take your concerns seriously, and we'll investigate the matter to ensure better communication and resolution moving for